# get predictions from final models

In [1]:
import gc
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses
from keras.models import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras import metrics
import keras
import sys
import pickle
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

K.tensorflow_backend._get_available_gpus()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

path_to_configs = "../"
sys.path.append(path_to_configs)
from configs import *
from models import *

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
MTL_final_final_model = pickle.load(open(path_to_configs+path_to_final_chosen_models + "MTL/final.p", "rb" ) )

MTL_LAYER_IDX = 3

dset_dpaths = [("all_human", ""),("Mouse", "_intersection"), ("Blood_GSE63060","_intersection"), 
               ("Blood_GSE63061", "_intersection")]

In [20]:
def get_model_layers(model_file, num_layers):
    
    # note: need to define custom functions for model in order to load, but these don't actually get used
    model = keras.models.load_model(model_file, custom_objects={"ignorenans_mse": ignorenans_mse})
    
    # define new model that cuts off the last several layers
    newmodel = Model(inputs = model.input, outputs = model.layers[num_layers-1].output)
    
    # agian, need to specify these parameters, but they aren't used since we don't retrain the model
    opt = optimizers.adam()  
    newmodel.compile(optimizer=opt, loss= "mse")
    
    return newmodel

### Save transformations from individual runs 

In [32]:

for dset,dpath in dset_dpaths:
    print("******SAVING EMBEDDINGS FOR %s******" %dset)
    
    for data_to_transform in ["test", "train"]:
        print("---%s Samples----"%data_to_transform)

        if dpath == "_intersection":
            reps = [0]

            path_to_data = path_to_configs + path_to_ext_val_data_folder + "processed_intersection/%s/"%dset
            if data_to_transform=="train":
                Xs = [np.loadtxt("%sGE_PCA_train.txt"%(path_to_data))]
            else:
                Xs =[np.loadtxt("%sGE_PCA_test.txt"%(path_to_data))]

            path_to_mods = path_to_configs + path_to_ext_val_results + "intersecting_gene_models/%s/"%dset

        else:
            reps = range(100)

            if data_to_transform=="train":
                print("this is just the normal md-ad medoid embedding!")
                with h5py.File(path_to_configs + path_to_MDAD_data_folders+"ACT_MSBBRNA_ROSMAP_PCA.h5", 'r') as hf:
                    Xs =[ hf["ge_transformed"][:,:num_components].astype(np.float64)]
            else:
                path_to_data = path_to_configs + path_to_ext_val_data_folder + "processed/%s/"%dset
                if dset=="all_human":
                    dset_subnames = os.listdir(path_to_data + "ge_pca")
                    Xs = [np.loadtxt("%sge_pca/%s"%(path_to_data, f)) for f in dset_subnames]
                else:
                    Xs = [np.loadtxt("%sGE_PCA.txt"%(path_to_data))]

            path_to_mods = path_to_configs + final_models_save_path




        for rep in reps:

            if rep%10==0: print(rep)
                
            if dpath == "_intersection":
                path_to_model = path_to_mods + "models/MTL/%s/%i/200.hdf5"%(MTL_final_final_model,rep)

            if dpath == "":
                path_to_model = path_to_mods + "models/MTL/%s/%s/%i/200.hdf5"%(full_pca_dataset, MTL_final_final_model,rep)


            mod_to_layer = get_model_layers(path_to_model, MTL_LAYER_IDX)

            for j,X in enumerate(Xs):
                X_transformed = mod_to_layer.predict(X)

                if dset=="all_human" and data_to_transform=="test":
                    savepath = "model_transformations%s/%s/%s/MTL/%i/%s/"%(dpath,dset,dset_subnames[j], HIDDEN_LAYER, data_to_transform)
                else:
                    savepath = "model_transformations%s/%s/MTL/%i/%s/"%(dpath,dset,HIDDEN_LAYER, data_to_transform)
                savepath = path_to_ext_val_results + savepath
                if not os.path.isdir(savepath):
                    os.makedirs(savepath)
                np.savetxt("%s%s.txt"%(savepath, rep),X_transformed)


******SAVING EMBEDDINGS FOR all_human******
---test Samples----
0


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


---train Samples----
this is just the normal md-ad medoid embedding!
0
******SAVING EMBEDDINGS FOR Mouse******
---test Samples----
0
---train Samples----
0
******SAVING EMBEDDINGS FOR Blood_GSE63060******
---test Samples----
0
---train Samples----
0
******SAVING EMBEDDINGS FOR Blood_GSE63061******
---test Samples----
0
---train Samples----
0


### For MD-AD consensus model, get final values 
(For intersection-based models, we just use a single run)

In [36]:
medoid_assignments = pd.read_csv(path_to_configs + path_to_medoids_info)

In [61]:
def save_mdad_embedding(path_to_run_embeddings, medoid_assignments):
    embedding = []
    for i,row in medoid_assignments[1:].iterrows():
        run_embedding = np.loadtxt("%s%s.txt"%(path_to_run_embeddings, row["run"]))
        embedding.append(run_embedding[:,row["node_idx"]])
    embedding = np.vstack(embedding).T
    
    return(embedding)

In [64]:
for dset,dpath in dset_dpaths:
    if dpath=="" and dset != "all_human":
        savepath = "model_transformations/%s/MTL/1/test/"%(dset)
        consensus_embedding = save_mdad_embedding(savepath, medoid_assignments)
        np.savetxt("%sfinal.txt"%(savepath),consensus_embedding)
        
    if dpath=="" and dset=="all_human":
        for f in os.listdir("model_transformations/all_human/"):
            savepath = "model_transformations/all_human/%s/MTL/1/"%f
            savepath = path_to_ext_val_results + savepath
            for data_to_transform in ["test", "train"]:
                consensus_embedding = save_mdad_embedding(savepath+data_to_transform+"/", medoid_assignments)
                np.savetxt("%s%s/final.txt"%(savepath, data_to_transform),consensus_embedding)